# Visualization with Streamlit

In this step we will make use of streamlit library to build the plots of predictions created using regression and nbeats time series.

In [1]:
%%writefile 'src/streamlit-app.py'

import streamlit as st
import digitalhub as dh
import pandas as pd
import dateutil.parser as dparser

PROJECT_NAME = "parcheggi-nk-scheduler-digitalhubdev"
proj = dh.get_or_create_project(PROJECT_NAME)
print("created project {}".format(PROJECT_NAME))

prediction_nbeats = proj.get_dataitem("parking_prediction_nbeats_model")
prediction_sarimax_regression = proj.get_dataitem("parking_data_predicted_regression")

rdf1 = prediction_sarimax_regression.as_df();
rdf1['datetime'] = rdf1['datetime'].apply(dparser.parse,fuzzy=True)

rdf2 = prediction_nbeats.as_df()
rdf2.datetime = pd.to_datetime(rdf2.datetime, unit='ms')

st.write("""prediction Sarimax regression""")
st.line_chart(rdf1, x="datetime", y="predicted_mean")

st.write("""prediction Nbeats model""")
st.line_chart(rdf2, x="datetime", y="predicted_mean")


Writing src/streamlit-app.py


In [2]:
#!streamlit run src/streamlit-app.py --browser.gatherUsageStats false

In [3]:
import digitalhub as dh
import getpass as gt

PROJECT_NAME = "parcheggi-nk-scheduler-"+gt.getuser()
proj = dh.get_or_create_project(PROJECT_NAME)
print("created project {}".format(PROJECT_NAME))

created project parcheggi-nk-scheduler-digitalhubdev


In [4]:
func = proj.new_function(name="streamlit-viz-v3",
                         kind="container",
                         python_version="PYTHON3_10",
                         lang="python",
                         base_image="ghcr.io/scc-digitalhub/digitalhub-serverless/python-runtime:3.9-latest",
                         image="ghcr.io/scc-digitalhub/digitalhub-serverless/python-runtime:3.9-latest",
                         code_src="src/streamlit-app.py", args= ["run", "src/streamlit-app.py"], command= "streamlit")

In [8]:
# import pandas as pd
# from datetime import datetime
# import dateutil.parser as dparser
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

In [18]:
# prediction_sarimax_regression = proj.get_dataitem("parking_data_predicted_regression")
# rdf1 = prediction_sarimax_regression.as_df();
# rdf1.dtypes

predicted_mean    float64
parcheggio         object
datetime           object
dtype: object

In [12]:
# import dateutil.parser as dparser
# rdf1['datetime'] = rdf1['datetime'].apply(dparser.parse,fuzzy=True)
# rdf1.dtypes

predicted_mean           float64
parcheggio                object
datetime          datetime64[ns]
dtype: object

In [14]:
# prediction_nbeats = proj.get_dataitem("parking_prediction_nbeats_model")
# rdf2 = prediction_nbeats.as_df()
# rdf2.datetime = pd.to_datetime(rdf2.datetime, unit='ms')
# rdf2.dtypes

parcheggio                object
datetime          datetime64[ns]
predicted_mean           float64
dtype: object

## Build

In [20]:
#func.run(action="build", instructions=["pip install streamlit"])

In [ ]:
# func.spec.to_dict()

In [40]:
#fun.run(action="serve", command='streamlit run src/streamlit-app.py --browser.gatherUsageStats false', )

## Serve

In [16]:
#func.run(action="serve", service_ports=[{"port": 5002, "target_port": 8501}])

## Expose the service

To make it accessible from outside, we'll need to create an API gateway.

Go to the Kubernetes Resource Manager component (available from dashboard) and go to the API Gateways section. To expose a service it is necessary to define

name of the gateway
the service to expose
the endpoint where to publish
and the authentication method (right now only no authentication or basic authentication are available). in case of basic authentication it is necessary to specify Username and Password.
The platform by default support exposing the methods at the subdomains of services.<platform-domain>, where platform-domain is the domain of the platform instance.

<img src="streamlit.jpg">

Save and, after a few moments, you will be able to call the API at the address you defined! If you set Authentication to Basic, don't forget that you have to provide the credentials.